In [59]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import parse_UK_Data


dataset = "../datasets/UK10_1.txt"
meta_data, distance_data, station_data = parse_UK_Data(dataset)
station_coordinates = pd.read_csv('../datasets/UK10_1_coordinates.csv', encoding='utf-8')
points_coordinate = station_coordinates[["lat", "long"]].to_numpy()

api = "Tfmt6Sxyn-nfinSHUyUo3xmVDWdtwAUyh6C8Y3HTZhU"

route = [[0,1,2,3], [4,5,6,7,8,9]]

In [60]:
meta_data["F-C Empty (l/100km)"] = 12.5
meta_data["F-C Full (l/100km)"] = 15
meta_data

,Customer Amount,Vehicle Curb Weight(kg),Max Load(kg),Minimum Speed(km/h),Maximum Speed(km/h),F-C Empty (l/100km),F-C Full (l/100km)
0,10,6350,3650,20,90,12.5,15


In [61]:
def get_results(vehicles:list, distance_matrix:pd.DataFrame, demand_data:pd.DataFrame, meta_data:pd.DataFrame) -> pd.DataFrame:
    
    def _get_total_distance(vehicles:list, distance_matrix:pd.DataFrame) -> list:
        total_vehicle_distance = []
        for vehicle, vehicle_route in enumerate(vehicles):
            distance = 0
            for i in range(len(vehicle_route) - 1):
                distance += distance_matrix.iloc[vehicle_route[i]][vehicle_route[i+1]]
            total_vehicle_distance.append(distance/1e4)
        return total_vehicle_distance
    
    def _get_total_load(vehicles:list, demand_data:pd.DataFrame) -> list:
        total_vehicle_load = []
        for vehicle, vehicle_route in enumerate(vehicles):
            load = 0
            for i in range(len(vehicle_route) - 1):
                load += int(demand_data.iloc[vehicle_route[i]]["Demand(kg)"])
            total_vehicle_load.append(load)
        return total_vehicle_load
    
    def _get_estimated_fuel_consumption(vehicles:list, meta_data:pd.DataFrame, distance_matrix:pd.DataFrame) -> list:
        total_vehicle_fuel_consumption = []
        for vehicle, vehicle_route in enumerate(vehicles):
            fc = 0
            for i in range(len(vehicle_route) - 1):
                #Distance in 100km
                distance = distance_matrix.iloc[vehicle_route[i]][vehicle_route[i+1]]/1e5
                #Demand in kg
                load = int(demand_data.iloc[vehicle_route[i]]["Demand(kg)"])
                #Fuel consumption between nodes driving empty vehicle
                fuel_consumption_empty = distance * meta_data['F-C Empty (l/100km)']
                load_rate = load / float(meta_data['Max Load(kg)'])
                #Additional fuel consumption when adding load at from_index
                fuel_consumption_load = distance * load_rate * (meta_data['F-C Full (l/100km)'] - meta_data['F-C Empty (l/100km)'])
                fc += np.float(fuel_consumption_empty + fuel_consumption_load)
                
            total_vehicle_fuel_consumption.append(fc)
    
        return total_vehicle_fuel_consumption
        
    def _get_avg_estimated_fuel_conspumtion(vehicle_distances:list, vehicle_fc:list) -> list:
        return  [fc/(dist/10) for dist,fc in zip(vehicle_distances, vehicle_fc)]
    
    vehicle_distances = _get_total_distance(vehicles, distance_matrix)
    vehicle_loads = _get_total_load(vehicles, demand_data)
    vehicle_fc = _get_estimated_fuel_consumption(vehicles, meta_data, distance_matrix)
    vehicle_avg_fc = _get_avg_estimated_fuel_conspumtion(vehicle_distances, vehicle_fc)
    
    results = pd.DataFrame()
    results["Total distance (km)"] = np.array(vehicle_distances)
    results["Total load (kg)"] = np.array(vehicle_loads)
    results["Total Estimated Fuel Consumption (L)"] = vehicle_fc
    results["Avg Estimated Fuel Conspumtion (L/100km)"] = vehicle_avg_fc
    
    return results

In [62]:
get_results(route, distance_data, station_data, meta_data)

,Total distance (km),Total load (kg),Total Estimated Fuel Consumption (L),Avg Estimated Fuel Conspumtion (L/100km)
0,15.465,1535,19.930941,12.887773
1,34.094,2322,43.698961,12.817200


In [76]:
import requests
def generate_large_distance_matrix(coordinates, api_key=""):
    """
        Generates a distance matrix from locations with Here.com api, calculates the matrix as Nx15x100 where N is batches.
        
        --Input: Coordinates for the locations as a numpy array.
                    Max size api: 100x1 or 15x100
                    Max size this function: 100x100
        
        --Output: Distance matrix and error codes as DataFrame.
                    Error code: 0 = valid
                    Error code: 3 = computation error, don't trust the corresponding value.
    """
    
    def _send_matrix_request(origin_coordinates, dest_coordinates, api_key):
        origins = [{"lat":lat, "lng":lng} for lat, lng in origin_coordinates]
        dests = [{"lat":lat, "lng":lng} for lat, lng in dest_coordinates]
        matrix_request = {"transportMode": "truck", "origins":origins, "destinations":dests, "regionDefinition": {"type":"world"}, "matrixAttributes":["distances", "travelTimes"]}
        response = requests.post("https://matrix.router.hereapi.com/v8/matrix?async=false&apiKey=%s" % (api_key), json=matrix_request)
        return response
    
    def _build_distance_matrix(response):
        
        distance_matrix = list(response.json().get("matrix").get("distances"))
        travel_time_matrix = list(response.json().get("matrix").get("travelTimes"))
        if "errorCodes" in response:
            error_matrix = list(response.json().get("matrix").get("errorCodes"))
        else:
            error_matrix = [0 for val in distance_matrix]
        #matrix_distances_df = pd.DataFrame(matrix_distances)
        #matrix_error_df = pd.DataFrame(matrix_error_codes)

        return distance_matrix, travel_time_matrix, error_matrix
    
    # This is used for testing, the input data is a list of the alphabet
    def _build_distance_matrix_2(origin_coordinates, dest_coordinates):
        distance_matrix = []
        for o in origin_coordinates:
            for d in dest_coordinates:
                distance_matrix.append("%s,%s" % (o, d))

        return distance_matrix
    
    ## https://developers.google.com/optimization/routing/vrp#distance_matrix_api
    max_size = 15*15
    num_locations = len(coordinates)
    max_rows = max_size // num_locations
    quotient, rest = divmod(num_locations, max_rows)
    #print("q: %s r: %s" % (quotient, rest))
    distance_matrix = []
    travel_time_matrix = []
    error_matrix = []
    # Send q requests, returning max_rows rows per request.
    
    destinations = coordinates

    for i in range(quotient):
        origin_coordinates = coordinates[i * max_rows: (i + 1) * max_rows]
        response = _send_matrix_request(origin_coordinates, destinations, api_key)
        resp_distance_matrix, resp_travel_time_matrix, resp_error_matrix = _build_distance_matrix(response)
        distance_matrix += resp_distance_matrix
        travel_time_matrix += resp_travel_time_matrix
        error_matrix += resp_error_matrix
        #distance_matrix += _build_distance_matrix_2(origin_coordinates, destinations)
    
    if rest > 0:
        origin_coordinates = coordinates[quotient * max_rows: quotient * max_rows + rest]
        response = _send_matrix_request(origin_coordinates, destinations, api_key)
        resp_distance_matrix, resp_travel_time_matrix, resp_error_matrix = _build_distance_matrix(response)
        distance_matrix += resp_distance_matrix
        travel_time_matrix += resp_travel_time_matrix
        error_matrix += resp_error_matrix
        #distance_matrix += _build_distance_matrix_2(origin_coordinates, destinations)
    
    
    distance_matrix_df = pd.DataFrame(np.array(distance_matrix).reshape([num_locations, num_locations]))
    travel_time_matrix_df = pd.DataFrame(np.array(travel_time_matrix).reshape([num_locations, num_locations]))
    error_matrix_df = pd.DataFrame(np.array(error_matrix).reshape([num_locations, num_locations]))
    
    return distance_matrix_df, travel_time_matrix_df, error_matrix_df

In [77]:
distance_matrix_df, travel_time_matrix, error_matrix_df = generate_large_distance_matrix(points_coordinate, api)